In [4]:
import os
import sys
# 使用insert 0即只使用github，避免交叉使用了pip安装的abupy，导致的版本不一致问题
if sys.path[0] != os.path.abspath('../'):
    sys.path.insert(0, os.path.abspath('../'))
#打印出sys.path的个数
print(len(sys.path))
for path in sys.path:
    print(path)

11
d:\luoting\git\stock\abu
C:\Users\luoting01\AppData\Local\Programs\Python\Python39\python39.zip
C:\Users\luoting01\AppData\Local\Programs\Python\Python39\DLLs
C:\Users\luoting01\AppData\Local\Programs\Python\Python39\lib
C:\Users\luoting01\AppData\Local\Programs\Python\Python39
d:\luoting\git\stock\abu\venv

d:\luoting\git\stock\abu\venv\lib\site-packages
d:\luoting\git\stock\abu\venv\lib\site-packages\win32
d:\luoting\git\stock\abu\venv\lib\site-packages\win32\lib
d:\luoting\git\stock\abu\venv\lib\site-packages\Pythonwin


In [9]:
# 基础库导入，注意第一次运行时会比较慢，要做数据的解压等处理操作
from __future__ import print_function
from __future__ import division

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
    
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os
import sys
# 使用insert 0即只使用github，避免交叉使用了pip安装的abupy，导致的版本不一致问题
if sys.path[0] != os.path.abspath('../'):
    sys.path.insert(0, os.path.abspath('../'))
import abupy

In [10]:
abupy.env.enable_example_env_ipython()

enable example env will only read RomDataBu/csv
enable example env will only read RomDataBu/csv


In [11]:
from abupy import ABuSymbolPd
ABuSymbolPd.make_kl_df('usJD') is None

True

In [12]:
abupy.env.disable_example_env_ipython()
us_jd = ABuSymbolPd.make_kl_df('usJD')
# 再次开启沙盒环境，本节的示例都是在沙盒数据环境下运行
abupy.env.enable_example_env_ipython()
tail = None
if us_jd is not None:
    tail = us_jd.tail()
tail

disable example env
disable example env
HTTPConnectionPool(host='gp.baidu.com', port=80): Max retries exceeded with url: /stocks/stockkline?from=android&os_ver=21&format=json&vv=3.3.0&uid=&BDUSS=&cuid=whaqnecen0melyc6849sjizj10jcmo1q11x3v1yg&channel=default_channel&device=A0001&logid=1731404037&actionid=1731403971&device_net_type=wifi&period=day&stock_code=usJD&fq_type=front (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x000002224A4C8370>: Failed to resolve 'gp.baidu.com' ([Errno 11001] getaddrinfo failed)"))
Traceback (most recent call last):
  File "d:\luoting\git\stock\abu\venv\lib\site-packages\urllib3\connection.py", line 199, in _new_conn
    sock = connection.create_connection(
  File "d:\luoting\git\stock\abu\venv\lib\site-packages\urllib3\util\connection.py", line 60, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "C:\Users\luoting01\AppData\Local\Programs\Python\Python39\lib\socket.py", li

In [13]:
from abupy import AbuFactorBuyXD, BuyCallMixin

class AbuFactorBuyBreak(AbuFactorBuyXD, BuyCallMixin):
    """示例继承AbuFactorBuyXD完成正向突破买入择时类, 混入BuyCallMixin，即向上突破触发买入event"""
    def fit_day(self, today):
        """
        针对每一个交易日拟合买入交易策略，寻找向上突破买入机会
        :param today: 当前驱动的交易日金融时间序列数据
        :return:
        """
        # 今天的收盘价格达到xd天内最高价格则符合买入条件
        if today.close == self.xd_kl.close.max():
            # 生成买入订单, 由于使用了今天的收盘价格做为策略信号判断，所以信号发出后，只能明天买
            return self.buy_tomorrow()
        return None

In [14]:
from abupy import AbuBenchmark
from abupy import AbuCapital

# buy_factors 60日向上突破，42日向上突破两个因子
buy_factors = [{'xd': 60, 'class': AbuFactorBuyBreak}, 
               {'xd': 42, 'class': AbuFactorBuyBreak}]
benchmark = AbuBenchmark()
capital = AbuCapital(1000000, benchmark)

In [15]:
from abupy import ABuPickTimeExecute
orders_pd, action_pd, _ = ABuPickTimeExecute.do_symbols_with_same_factors(['usTSLA'],
                                                                            benchmark,
                                                                            buy_factors,
                                                                            None,
                                                                            capital, show=True)

unsupported operand type(s) for &: 'DatetimeArray' and 'DatetimeArray'
Traceback (most recent call last):
  File "d:\luoting\git\stock\abu\abupy\AlphaBu\ABuPickTimeExecute.py", line 117, in _batch_symbols_with_same_factors
    kl_pd = kl_pd_manager.get_pick_time_kl_pd(target_symbol)
  File "d:\luoting\git\stock\abu\abupy\TradeBu\ABuKLManager.py", line 169, in get_pick_time_kl_pd
    kl_pd = self._fetch_pick_time_kl_pd(target_symbol)
  File "d:\luoting\git\stock\abu\abupy\TradeBu\ABuKLManager.py", line 157, in _fetch_pick_time_kl_pd
    return ABuSymbolPd.make_kl_df(target_symbol, data_mode=EMarketDataSplitMode.E_DATA_SPLIT_UNDO,
  File "d:\luoting\git\stock\abu\abupy\MarketBu\ABuSymbolPd.py", line 305, in make_kl_df
    df, _ = _make_kl_df(symbol, data_mode=data_mode,
  File "d:\luoting\git\stock\abu\abupy\MarketBu\ABuSymbolPd.py", line 128, in _make_kl_df
    df = _benchmark(df, benchmark, temp_symbol)
  File "d:\luoting\git\stock\abu\abupy\MarketBu\ABuSymbolPd.py", line 48, in _bench